In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split, get_json_object, json_tuple, from_json
from pyspark.sql.types import *
import json

spark = SparkSession.builder.appName("Wordle score streaming").getOrCreate()


lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9008) \
    .load()

# Split the lines into words
# words = lines.select(
#    explode(
#        split(lines.value, 'Wordle 222 ')
#    ).alias("word")
# )


schema = StructType(). \
    add('id', LongType(), False). \
    add('created_at', StringType(), False) .\
    add('user', StructType().add("id_str",StringType(), False), False). \
    add('text', StringType(), False)


filtered_data = lines \
    .selectExpr('CAST(value AS STRING)') \
    .select(from_json('value', schema).alias('tweet_data')) \
    .select('tweet_data.id', 'tweet_data.created_at', 'tweet_data.user.id_str', 'tweet_data.text')


df.printSchema()


# # Generate running word count
# wordCounts = words.groupBy("word").count()



# Start running the query that prints tweet data to the console
query = filtered_data \
    .writeStream \
    .queryName("word_counts") \
    .format("console") \
    .outputMode("append") \
    .start()

query.awaitTermination()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/03 16:15:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/03 16:15:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/03 16:15:20 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


root
 |-- id: long (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id_str: string (nullable = true)
 |-- text: string (nullable = true)



22/02/03 16:15:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/_p/fncz_94x46n3hpt2_kbtc8540000gn/T/temporary-b95acc8a-7441-4ace-b7fa-51acb4dfdcf8. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/02/03 16:15:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+----------+------+----+
| id|created_at|id_str|text|
+---+----------+------+----+
+---+----------+------+----+



22/02/03 16:15:28 WARN TextSocketMicroBatchStream: Stream closed by localhost:9008


-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+--------------------+---------+--------------------+
|                 id|          created_at|   id_str|                text|
+-------------------+--------------------+---------+--------------------+
|1489256188583690240|Thu Feb 03 15:15:...|178582264|Why isn’t Trump i...|
+-------------------+--------------------+---------+--------------------+



In [ ]:
lines.printSchema()